In [20]:
import pandas as pd
import numpy as np

kcc = pd.read_csv('D:\Flation\mod_2\Project2\Mod_2_Project/Mod_2 _Project/data_files/kc_cleaned.csv')

continuous = ['price', 'sqft_living', 'sqft_lot']
categoricals = ['floors', 'condition', 'waterfront', 'grade', 'zipcode', 'sale_month']

kcc_cont = kcc[continuous]

# log features
log_names = [f'{column}_log' for column in kcc_cont.columns]

kcc_log = np.log(kcc_cont)
kcc_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

kcc_log_norm = kcc_log.apply(normalize)

# one hot encode categoricals
kcc_ohe = pd.get_dummies(kcc[categoricals], columns=['floors','condition', 'waterfront', 'zipcode', 'grade', 'sale_month'], drop_first=True)

preprocessed = pd.concat([kcc_log_norm, kcc_ohe], axis=1)

In [21]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [24]:
X = preprocessed.drop('price_log', axis=1)
y = preprocessed['price_log']

result = stepwise_selection(X, y, verbose = True)
print('resulting features:')
print(result)

C:\Users\eduar\anaconda3\envs\learn-env\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Add  grade_11                       with p-value 0.0
Add  grade_9                        with p-value 0.0
Add  sqft_living_log                with p-value 0.0
Add  grade_10                       with p-value 2.53439e-242
Add  zipcode_98023                  with p-value 7.42636e-189
Add  zipcode_98004                  with p-value 7.47526e-189
Add  zipcode_98042                  with p-value 1.48077e-146
Add  grade_8                        with p-value 4.44609e-143
Add  grade_12                       with p-value 5.23727e-159
Add  zipcode_98092                  with p-value 2.30842e-134
Add  zipcode_98112                  with p-value 1.37787e-127
Add  waterfront_1.0                 with p-value 5.64348e-134
Add  zipcode_98115                  with p-value 5.64269e-134
Add  zipcode_98103                  with p-value 3.82025e-138
Add  zipcode_98117                  with p-value 2.16061e-140
Add  zipcode_98105                  with p-value 2.09746e-119
Add  zipcode_98003                 

C:\Users\eduar\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


Add  zipcode_98045                  with p-value 3.42106e-18
Drop zipcode_98038                  with p-value 0.309548
Add  zipcode_98146                  with p-value 3.75544e-22
Drop zipcode_98055                  with p-value 0.454953
Add  zipcode_98019                  with p-value 2.68456e-27
Drop zipcode_98058                  with p-value 0.507672
Add  zipcode_98070                  with p-value 4.79179e-12
Add  zipcode_98014                  with p-value 1.31586e-11
Add  zipcode_98038                  with p-value 3.48465e-08
Drop zipcode_98188                  with p-value 0.0551127
Add  zipcode_98178                  with p-value 5.06615e-08
Drop zipcode_98198                  with p-value 0.384892
Add  zipcode_98058                  with p-value 4.32442e-10
Drop zipcode_98031                  with p-value 0.397006
Add  zipcode_98010                  with p-value 1.62385e-10
Drop zipcode_98022                  with p-value 0.695684
Add  zipcode_98055                  with p-v

In [25]:
import statsmodels.api as sm
X_fin = X[result]
X_with_intercept = sm.add_constant(X_fin)
model = sm.OLS(y,X_with_intercept).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              price_log   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     1581.
Date:                Thu, 22 Oct 2020   Prob (F-statistic):               0.00
Time:                        12:32:08   Log-Likelihood:                -7259.7
No. Observations:               19221   AIC:                         1.469e+04
Df Residuals:                   19135   BIC:                         1.537e+04
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -1.5214      0.030    -51.549      0.000      -1.579      -1.464
grade_11            1.1422      0.025     45.357      0.000       1.093       1.192
grade_9             0.6478      0.015     42.991      0.000       0.618       0.677
sqft_living_log     0.3668      0.004     86.085      0.000       0.358       0.375
grade_10            0.8662      0.019     46.400      0.000       0.830       0.903
zipcode_98023      -0.1481      0.019     -7.892      0.000      -0.185      -0.111
zipcode_98004       2.0710      0.023     89.894      0.000       2.026       2.116
grade_8             0.3614      0.012     29.833      0.000       0.338       0.385
grade_12            1.4687      0.044     33.634      0.000       1.383       1.554
zipcode_98092      -0.0651      0.022     -2.984      0.003      -0.108      -0.022
zipcode_98112       1.9717      0.025     79.572      0.000       1.923       2.020
waterfront_1.0      1.3084      0.030     43.113      0.000       1.249       1.368
zipcode_98115       1.5130      0.018     83.762      0.000       1.478       1.548
zipcode_98103       1.5604      0.018     84.488      0.000       1.524       1.597
zipcode_98117       1.5061      0.018     81.620      0.000       1.470       1.542
zipcode_98105       1.7735      0.026     67.623      0.000       1.722       1.825
zipcode_98003      -0.0689      0.024     -2.905      0.004      -0.115      -0.022
zipcode_98040       1.6216      0.025     65.762      0.000       1.573       1.670
zipcode_98199       1.6033      0.023     68.815      0.000       1.558       1.649
zipcode_98119       1.8787      0.029     64.001      0.000       1.821       1.936
zipcode_98033       1.4114      0.020     69.859      0.000       1.372       1.451
zipcode_98107       1.6180      0.025     64.206      0.000       1.569       1.667
zipcode_98039       2.4155      0.054     44.750      0.000       2.310       2.521
zipcode_98116       1.4363      0.023     63.768      0.000       1.392       1.480
zipcode_98122       1.5272      0.024     63.441      0.000       1.480       1.574
zipcode_98002      -0.1200      0.028     -4.308      0.000      -0.175      -0.065
zipcode_98109       1.8904      0.038     50.374      0.000       1.817       1.964
zipcode_98032      -0.1585      0.034     -4.651      0.000      -0.225      -0.092
condition_3         0.2851      0.027     10.451      0.000       0.232       0.339
zipcode_98102       1.7919      0.039     45.900      0.000       1.715       1.868
grade_13            2.0852      0.101     20.618      0.000       1.887       2.283
sqft_lot_log        0.1199      0.004     32.348      0.000       0.113       0.127
grade_7             0.1429      0.010     14.127      0.000       0.123       0.163
zipcode_98144       1.2663      0.023     56.245      0.000

In [26]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(X, y.values.ravel()) 
selector.support_ 

array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False])

In [27]:
selected_columns = X.columns[selector.support_ ]
linreg.fit(X[selected_columns],y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [32]:
selected_columns

Index(['waterfront_1.0', 'zipcode_98004', 'zipcode_98039', 'grade_12',
       'grade_13'],
      dtype='object')

In [28]:
yhat = linreg.predict(X[selected_columns])

In [29]:
SS_Residual = np.sum((y-yhat)**2)
SS_Total = np.sum((y-np.mean(y))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X[selected_columns].shape[1]-1)

In [30]:
r_squared

0.13049542582392004

In [34]:
adjusted_r_squared

0.13026916910412412

In [ ]:
mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test = np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

In [35]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)

In [36]:
from sklearn.metrics import mean_squared_error
test_residuals = y_hat_test - y_test

test_mse = mean_squared_error(y_test, y_hat_test)
test_mse

34223272525.68238

In [ ]:
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.stats.api as sms


results = []
for idx, column in enumerate(data.columns):
    print (f"Ames Housing DataSet - Regression Analysis and Diagnostics for SalePrice~{column}")
    print ("-------------------------------------------------------------------------------------")

    f = f'SalePrice~{column}'
    model = smf.ols(formula=f, data=data).fit()
    
    fig, axes = plt.subplots(figsize=(15,12))
    fig = sm.graphics.plot_regress_exog(model, column, fig=fig)
    fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)
    fig.tight_layout()
    plt.show()
    
    results.append([column, model.rsquared, model.params[0], model.params[1], model.pvalues[1], sms.jarque_bera(model.resid)[0]])
    input("Press Enter to continue...")